##### QnA chatcot using RAG. context is standford lectures. this is a take home assignment for an application(Ema)

In [1]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=getpass.getpass()

 ········


# Data Ingestion

In [2]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader1=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/introduction/",),
                    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap")))) 
loader2=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/legality/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap"))))
loader3=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/modeling/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main_content-wrap"))))
loader4=WebBaseLoader(web_paths=("https://stanford-cs324.github.io/winter2022/lectures/training/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("main-content-wrap"))))
### we are using 4 lectures: intro, legality, modeling and training. 
### since all the content was inside the main class "main-content-wrap" we used that


In [45]:
loaders=[loader1,loader2,loader3,loader4] #array of loaders to work upon

In [46]:
docs=[]
for loader in loaders:
    docs.extend(loader.load()) #loading documents together

# Transform

In [48]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [49]:
split=splitter.split_documents(docs) # splitting documents into chunks 

In [50]:
len(split)

74

In [51]:
split[0].page_content

'LecturesIntroduction \\[\\newcommand{\\sV}{\\mathcal{V}} \\newcommand{\\nl}[1]{\\textsf{#1}} \\newcommand{\\generate}[1]{\\stackrel{#1}{\\rightsquigarrow}}\\]Welcome to CS324! This is a new course on understanding and developing large language models.What is a language model?A brief historyWhy does this course exist?Structure of this course  What is a language model?The classic definition of a language model (LM) is a probability distribution over sequences of tokens. Suppose we have a vocabulary \\(\\sV\\) of a set of tokens. A language model \\(p\\) assigns each sequence of tokens \\(x_1, \\dots, x_L \\in \\sV\\) a probability (a number between 0 and 1):\\[p(x_1, \\dots, x_L).\\]The probability intuitively tells us how “good” a sequence of tokens is. For example, if the vocabulary is \\(\\sV = \\{ \\nl{ate}, \\nl{ball}, \\nl{cheese}, \\nl{mouse}, \\nl{the} \\}\\), the language model might assign (demo):\\[p(\\nl{the}, \\nl{mouse}, \\nl{ate}, \\nl{the}, \\nl{cheese}) = 0.02,\\] \\[p(

# Retriever

In [52]:
from langchain_core.prompts import ChatPromptTemplate

system_message=(""" You are a question-answer assistant.
                    Answer the questions based on the context provided.
                    Keep the answers as accurate and concise as possible.
                    If the answer is not present in the context then say its not available in the context.
                    \n\n
                    {context}.""")

prompt=ChatPromptTemplate.from_messages([("system",system_message),
                                        ("human","{input}")])


In [53]:
from langchain_community.llms import Ollama

model=Ollama(model="llama3")

In [54]:
from langchain.chains.combine_documents import create_stuff_documents_chain

stuff_doc_chain=create_stuff_documents_chain(model,prompt) #creating stuff document chain or basically chaining llm and prompt

In [64]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_fireworks import FireworksEmbeddings
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS

vector_storage=Chroma.from_documents(split,OllamaEmbeddings(model="nomic-embed-text"), persist_directory="vector_store",collection_name="qna_embeddings") #initializing our vector storage

In [65]:
vector_storage

In [66]:
retriever=vector_storage.as_retriever()

In [67]:
from langchain.chains import create_retrieval_chain

retriever_chain=create_retrieval_chain(retriever,stuff_doc_chain)

In [68]:
response=retriever_chain.invoke({"input":"What are some milestone model architectures and papers in the last few years?"})

In [69]:
response["answer"]

'Based on the context provided, I\'ll answer your question about milestone model architectures and papers in the last few years.\n\nIn recent years, several breakthroughs have been made in neural language modeling. Here are a few milestones:\n\n1. **Recurrent Neural Networks (RNNs)**: Introduced by Sepp Hochreiter et al. in their 1997 paper "Long Short-Term Memory," RNNs allowed the conditional distribution of a token to depend on the entire context.\n\n2. **Transformers**: In 2017, Ashish Vaswani et al. introduced the Transformer architecture for machine translation, which revolutionized sequence-to-sequence tasks and set the stage for further advancements in language modeling.\n\n3. **BERT (Bidirectional Encoder Representations from Transformers)**: Released by Jacob Devlin et al. in 2018, BERT is a pre-trained language model that has achieved state-of-the-art results on a wide range of natural language processing (NLP) tasks.\n\n4. **GPT-3 (Generative Pre-training of Transformers-3)

In [81]:
# response

In [78]:
for docs in response["context"]:
    print(docs.metadata["source"])

https://stanford-cs324.github.io/winter2022/lectures/introduction/
https://stanford-cs324.github.io/winter2022/lectures/introduction/
https://stanford-cs324.github.io/winter2022/lectures/introduction/
https://stanford-cs324.github.io/winter2022/lectures/introduction/


In [77]:
response2=retriever_chain.invoke({"input":"what are the types of law?"})

In [79]:
response2["answer"]

'According to the provided context, there are three main types of law mentioned:\n\n1. Common law (judiciary): Also known as case law, common law is based on judges referencing previous similar cases and making a ruling (precedent).\n2. Statutory law (legislature): Also known as written law, statutory law is produced by government agencies through the legislative process (e.g., congress passing a bill).\n3. Regulatory law (executive): Also known as administrative law, this is law that is created by the executive branch of government, often focusing on procedures.\n\nThese types are mentioned in the context to provide an overview of different sources and forms of law.'

In [80]:
for docs in response2["context"]:
    print(docs.metadata["source"])

https://stanford-cs324.github.io/winter2022/lectures/legality/
https://stanford-cs324.github.io/winter2022/lectures/legality/
https://stanford-cs324.github.io/winter2022/lectures/legality/
https://stanford-cs324.github.io/winter2022/lectures/legality/


# Putting everything at one place

In [ ]:
class StanfordLectureBot:
    def __init__(self):
        